In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
import matplotlib.patches as mpatches

plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

# Boxes and CHSH-value

In [ ]:
def P_L(mu, nu, sigma, tau):
    new_box = np.zeros((4,4))
    for a in range(2):
        for b in range(2):
            for x in range(2):
                for y in range(2):
                    if a==(mu*x+nu)%2 and b==(sigma*y+tau)%2:
                        new_box[2*x+y, 2*a+b] = 1         
    return new_box


def P_NL(mu, nu, sigma):
    new_box = np.zeros((4,4))
    for a in range(2):
        for b in range(2):
            for x in range(2):
                for y in range(2):
                    if (a+b)%2==(x*y + mu*x + nu*y + sigma)%2:
                        new_box[2*x+y, 2*a+b] = 0.5               
    return new_box


PR = P_NL(0,0,0)
PRbar = P_NL(0,0,1)
PRprime = P_NL(1,1,1)
P_0 = P_L(0,0,0,0)
P_1 = P_L(0,1,0,1)
SR = (P_0 + P_1)/2
I = 0.25*np.ones((4,4))
SRbar = 2*I-SR


def corNLB(p):
    return p*PR + (1-p)*SR

In [ ]:
CHSH = np.zeros((4,4))
CHSHprime = np.zeros((4,4))

for a in range(2):
    for b in range(2):
        for x in range(2):
            for y in range(2):
                if (a+b)%2==(x*y)%2:
                    CHSH[2*x+y, 2*a+b] = 0.25  
                if (a+b)%2==((x+1)*(y+1))%2:
                    CHSHprime[2*x+y, 2*a+b] = 0.25 

def CHSHvalue(Box):
    return np.sum(Box*CHSH)

def CHSHprimevalue(Box):
    return np.sum(Box*CHSHprime)

# Definition of $\mathcal Q$

We use this article to draw $\mathcal Q$: https://arxiv.org/abs/quant-ph/0309137

In [ ]:
def E(Box, x, y):
    return Box[2*x+y, 2*0+0] + Box[2*x+y, 2*1+1] - Box[2*x+y, 2*0+1] - Box[2*x+y, 2*1+0]

def is_in_Q(Box1, Box2, Box3, alpha, beta):
    # alpha, beta, gamma = Compute_Coeff(PR, SR, I, CHSHprime_value=CHSHprime_value, CHSH_value=CHSH_value)
    Box = alpha*Box1 + beta*Box2 + (1-alpha-beta)*Box3
        # Box is a 4x4 matrix
    coeff00 = np.arcsin(E(Box,0,0))
    coeff01 = np.arcsin(E(Box,0,1))
    coeff10 = np.arcsin(E(Box,1,0))
    coeff11 = np.arcsin(E(Box,1,1))
    return coeff00 + coeff01 + coeff10 - coeff11 <= np.pi

In [ ]:
def draw_Q(Box1, Box1name, Box2, Box2name, Box3, Box3name, precision=0.001):

    # Draw the initial background triangle:
    x1, y1 = CHSHprimevalue(Box1), CHSHvalue(Box1)
    x2, y2 = CHSHprimevalue(Box2), CHSHvalue(Box2)
    x3, y3 = CHSHprimevalue(Box3), CHSHvalue(Box3)
    triangle = plt.Polygon(np.array([[x1, y1], [x2,y2], [x3, y3]]), color=hsv_to_rgb([0.475, 1, 0.9]))
    plt.gca().add_patch(triangle)

    Quantum_color = 'pink'
    beta=1
    alpha=0
    alpha_memory=0
    while beta>0:
        if is_in_Q(Box1, Box2, Box3, alpha, beta):
            Box = alpha*Box1 + beta*Box2 + (1-alpha-beta)*Box3
            plt.plot([CHSHprimevalue(Box), CHSHprimevalue(Box)], [CHSHvalue(Box), CHSHvalue(Box3)], color=Quantum_color)
            alpha_memory=alpha
            beta-=precision
            alpha=1-beta
        else:
            alpha-=precision
            if alpha<0:
                beta-=precision
                alpha=1-beta

    Box = alpha_memory*Box1 + (1-alpha_memory)*Box3
    triangle2 = plt.Polygon(np.array([[CHSHprimevalue(Box), CHSHvalue(Box)], [CHSHprimevalue(Box),CHSHvalue(Box3)], [CHSHprimevalue(Box3), CHSHvalue(Box3)]]), color=Quantum_color)
    plt.gca().add_patch(triangle2)

    plt.text(x1+0.001, y1, Box1name, horizontalalignment='center',weight="bold")
    plt.text(x2, y2+0.007, Box2name, verticalalignment='center',weight="bold")
    plt.text(x3, y3-0.007, Box3name, verticalalignment='center',weight="bold")
    plt.title("Quantum area drawn using [Masanes03]")
    plt.xlabel("$\mathbb{P}($win at CHSH'$)$")
    plt.ylabel("$\mathbb{P}($win at CHSH$)$")
    handles = []
    handles.append( mpatches.Patch(color=hsv_to_rgb([0.475, 1, 0.8]), label='$\mathcal{N\!S}$-boxes.') )
    handles.append( mpatches.Patch(color=Quantum_color, label='Quantum boxes.') )
    plt.legend(handles=handles, loc='upper right', fontsize = 10)
    plt.show()
        

# Draw $\mathcal Q$

In [ ]:
is_in_Q(PR, SR, (2/3)*PR+(1/3)*SRbar, 0.2, 0)

In [ ]:
draw_Q(PR, "PR", SR, "SR", (2/3)*PR+(1/3)*SRbar, "⅔PR + ⅓SRbar", precision=0.001)